In [17]:
import os
import json
from tqdm import tqdm

In [19]:
question = "<image>\nFrom this aerial image of an urban street scene, identify and trace all visible road markings, including lane dividers, lane boundaries, bike lanes. For each marking, output a polyline or a sequence of (x, y) pixel coordinates representing its shape. Only include visible markings painted on the road surface."
path = '/data_ssd/lei/OpenSatMap_VLM/train/'
regions = sorted(os.listdir(path))
annotations = []
for region in tqdm(regions):
    imgs = [f for f in sorted(os.listdir(path + region)) if f.endswith(".png")]
    for idx, img in enumerate(imgs):
        anns = {}
        id = region + '_' + str(idx).zfill(4)
        img_path = path + '{}/{}'.format(region, img)
        json_path = img_path.split('.')[0] + '.json'
        with open(json_path, 'r') as f:
            lines = json.load(f)
        ans = ''
        for line in lines:
            round_line = [[round(x), round(y)] for x, y in line]
            if any(x < 0 or x > 512 or y < 0 or y > 512 for x, y in round_line):
                assert 0
            ans = ans + "<line> " + " ".join(f"<{x}> <{y}>" for x, y in round_line) + " </line> "
        
        anns["id"] = id
        anns["images"] = img_path
        anns["conversations"] = []
        anns["conversations"].append({"role": "user", "content": question})
        anns["conversations"].append({"role": "assistant", "content": ans})
        annotations.append(anns)


with open("/data_ssd/lei/OpenSatMap_VLM/annotations.json", "w", encoding="utf-8") as f:
    json.dump(annotations, f, indent=4, ensure_ascii=False)

100%|██████████| 606/606 [00:19<00:00, 30.84it/s]


In [22]:
with open("/data_ssd/lei/OpenSatMap_VLM/annotations_val.json", "w", encoding="utf-8") as f:
    json.dump(annotations[310000:], f, indent=4, ensure_ascii=False)

In [20]:
len(annotations)


316357